In [36]:
import json
import tqdm
from flair.data import Sentence

In [37]:
def read_data(input_file_path):
    data = []

    with open(input_file_path) as f:
        for line in f:
            data.append(json.loads(line))

    return data

In [51]:
def write_data(output_file_path, data):
    # Open a new JSON file for writing
    with open(output_file_path, "w") as output_file:
        for data_line in data:
            output_file.write(json.dumps(data_line) + "\n")

    print(f"Data with 'Test' property added has been saved to {output_file_path}")

In [1170]:
data = read_data("./squad_data_validation_pos_ner.json")

In [1159]:
def find_start_position(text, pos_context, answer, context):
    # print(text)
    sentence = Sentence(text)
    tokens = []
    spaces_no = context[:answer].count(" ")
    # pos_context = pos_context.replace('\\','')

    for word in sentence:
        tokens.append(word.text)
    # print(tokens)

    start_position = 0
    token_number = 0
    match = False
    no_match = -1
    unicode = False
    for start_position in range(len(pos_context) - len(tokens) + 1):
        k_words_context_ = [bytes(list(line.values())[0][0], "utf-8").decode("unicode_escape") for line in
                      pos_context[start_position:start_position + len(tokens)]]
        k_words_context = [list(line.values())[0][0].replace('\\','') for line in
                      pos_context[start_position:start_position + len(tokens)]]
        next_word = [list(line.values())[0][0] for line in
                      pos_context[start_position + len(tokens):start_position + len(tokens) + 1]]
        # print(k_words_context)

        if context.count(text) == 1 and k_words_context == tokens:
            match = True
            break

        if k_words_context == tokens and (token_number + spaces_no == answer or token_number + spaces_no + 1 == answer):
            match = True
            break

        # if (token_number + spaces_no == answer):
        #     no_match = start_position
        #     print('HI 1')
        #     break
        elif next_word == ' ' and token_number + spaces_no + 1 == answer:
            no_match = start_position
            break
        if "'" in k_words_context_:
            token_number += len(k_words_context_[0]) + 1
        else:
            token_number += len(k_words_context_[0])
    if match:
        return start_position
    return no_match

In [1161]:
special_characters = ["$", ",", "''", "-LRB-", "-RRB-", ".", ":", "``"]


def get_proc_pos_between_item(pos_context, context, answers):
    positions = {}
    copy_answer = []
    i = 0
    for answer in answers["answer_start"]:
        answer_position = find_start_position(answers["text"][i], pos_context, answer, context)
        if answer_position != -1:
            positions[answer_position] = answers["answer_start"][i]
            copy_answer.append(answers["answer_start"][i])
        i += 1
    pos = ""
    offset = 0
    offsets = {}
    for index in range(len(pos_context)):
        pos_text = pos_context[index][str(index)]

        if pos_text[1] in special_characters:
            pos_text[1] = "sym"
        if index in positions.keys():
            offsets[positions[index]] = offset
        pos += f"{pos_text[0]} [{pos_text[1].lower()}] "
        offset += len(f" [{pos_text[1]}] ")

    answer_starts = []

    for index in range(len(copy_answer)):
        answer = copy_answer[index]
        spaces_no = context[:answer].count(" ") + 1
        answer += offsets[answer] - spaces_no
        if pos[answer : answer + 1] == " ":
            answer += 1
        # if len(answers["text"]) > index:
        #     if len(answers["text"][index]) > 1:
        #         if pos[answer][0] == answers["text"][index][1]:
        #             answer -= 1
        answer_starts.append(answer)

    return pos, answer_starts


In [1162]:
example_id = 1 

pos, answer_starts = get_proc_pos_between_item(data[example_id]['POS_context'], data[example_id]['context'],
                          data[example_id]['answers'])
pos

'Super [nnp] Bowl [nnp] 50 [cd] was [vbd] an [dt] American [jj] football [nn] game [nn] to [to] determine [vb] the [dt] champion [nn] of [in] the [dt] National [nnp] Football [nnp] League [nnp] ( [hyph] NFL [nnp] ) [sym] for [in] the [dt] 2015 [cd] season [nn] . [sym] The [dt] American [nnp] Football [nnp] Conference [nnp] ( [sym] AFC [nnp] ) [sym] champion [nn] Denver [nnp] Broncos [nnps] defeated [vbd] the [dt] National [nnp] Football [nnp] Conference [nnp] ( [sym] NFC [nnp] ) [sym] champion [nn] Carolina [nnp] Panthers [nnps] 24 [cd] – [sym] 10 [cd] to [to] earn [vb] their [prp$] third [jj] Super [nnp] Bowl [nnp] title [nn] . [sym] The [dt] game [nn] was [vbd] played [vbn] on [in] February [nnp] 7 [cd] , [sym] 2016 [cd] , [sym] at [in] Levi [nnp] \'s [pos] Stadium [nnp] in [in] the [dt] San [nnp] Francisco [nnp] Bay [nnp] Area [nnp] at [in] Santa [nnp] Clara [nnp] , [sym] California [nnp] . [sym] As [in] this [dt] was [vbd] the [dt] 50th [jj] Super [nnp] Bowl [nnp] , [sym] the [dt] 

In [1196]:
for item in tqdm.tqdm(data):
    context = item['context']
    question = item['question']
    item['context'], item['answers']['answer_start'] = get_proc_pos_between_item(item['POS_context'], context, item['answers'])

    item.pop('POS_context', None)
    item.pop('POS_question', None)
    item.pop('NER_question', None)
    item.pop('NER_context', None)

100%|██████████| 10570/10570 [00:16<00:00, 622.62it/s]


In [1195]:
incorrect_answers_no = 0

for index, line in tqdm.tqdm(enumerate(data)):
    try:
        if len(line["answers"]["answer_start"]) == 0:
            continue
        answer_start = line["answers"]["answer_start"][0]
        answer = line["answers"]["text"][0]

        if answer_start == -1:
            continue

        if line["context"][answer_start:][0] != answer[0]:
            # line["answers"]["answer_start"][0] -= 1
            incorrect_answers_no += 1
            #print(index, line["context"][answer_start:], 4*'-', answer, answer_start)

    except Exception as e: 
        print(e)

10570it [00:00, 250941.26it/s]


In [1197]:
write_data("./squad_data_validation_pos_between.json", data)

Data with 'Test' property added has been saved to ./squad_data_validation_pos_between.json


In [1264]:
proc_data = read_data('./squad_data_validation_pos_between.json')

In [1266]:
def rreplace(string, substring, replacement):
    k = string.rfind(substring)
    return string[:k] + replacement + string[k+len(substring):]

In [1268]:
symbols = ["'", '.', ',', ')']


def get_updated_answer(context, answers):
    new_answers = []
    if len(answers['answer_start']) == 0:
        return

    for index, answer in enumerate(answers['text']):
        new_answer = ''
        if len(answers['answer_start']) < index + 1:
            return new_answers
        answer_start = answers['answer_start'][index]
        if answer_start != -1:
            tokenized_context = Sentence(context[answer_start:])
            no_of_words = len(Sentence(answer))
            max_length = no_of_words * 3 + no_of_words
            if max_length > len(tokenized_context):
                max_length = len(tokenized_context)
            minus_words = 0
            for token_index in range(max_length):
                # if token_index == max_length - minus_words - 1:
                #     break
                if (tokenized_context[token_index].text == '[' and tokenized_context[token_index+1].text != '[') or (len(tokenized_context) > token_index+1 and tokenized_context[token_index].text!=']' and tokenized_context[token_index+1].text==']') or token_index == no_of_words * 4 - 1:
                    new_answer += tokenized_context[token_index].text
                elif tokenized_context[token_index].text in symbols and tokenized_context[token_index+1].text != '[':
                    new_answer += tokenized_context[token_index].text
                elif (tokenized_context[token_index].text.replace('.', '').isalpha() or tokenized_context[token_index].text[-1] in '."') and tokenized_context[token_index+1].text in '.$]':
                    new_answer += tokenized_context[token_index].text
                    # minus_words += 1
                elif len(tokenized_context) > token_index+1 and tokenized_context[token_index].text.replace('.', '').isnumeric() and tokenized_context[token_index+1].text in '.$]':
                    new_answer += tokenized_context[token_index].text
                elif tokenized_context[token_index].text.replace('-', '').isalpha() and tokenized_context[token_index+1].text in '.$]':
                    new_answer += tokenized_context[token_index].text
                elif len(tokenized_context) > token_index+1 and tokenized_context[token_index+1].text[0].isnumeric() and tokenized_context[token_index].text in '-':
                    new_answer += tokenized_context[token_index].text
                else:
                    new_answer += tokenized_context[token_index].text + ' '
        new_answer = new_answer.rstrip()
        if answer_start != -1:
            tokens_new_answer = Sentence(new_answer)
            # print(tokens_new_answer[-2])
            if tokens_new_answer[-2].text == '[':
                # print(tokens_new_answer[-2])
                new_answer += ']'
            if tokens_new_answer[-1].text == '[':
                #print(tokens_new_answer[-1], tokens_new_answer[-2].text)
                new_answer = rreplace(new_answer, tokens_new_answer[-1].text, '')
                new_answer = rreplace(new_answer, tokens_new_answer[-2].text, '')
                new_answer = new_answer.rstrip()
        new_answers.append(new_answer)
    return new_answers

In [1269]:
example_id = 4892

In [1275]:
get_updated_answer(proc_data[example_id]['context'], proc_data[example_id]['answers'])

['dial-up [jj] terminal [nn] to [in] a [dt] PAD [nn] , [sym] or [cc] , [sym] by [in] linking [vbg] a [dt] permanent [jj] X.25 [nn] node [nn] to [in] the [dt] network. [nn] [ [sym] citation [nn] needed [vbn] ] [sym]',
 'dial-up [jj] terminal [nn] to [in] a [dt] PAD [nn] , [sym] or [cc] , [sym] by [in] linking [vbg] a [dt] permanent [jj] X.25 [nn] node [nn]']

In [1276]:
for item in tqdm.tqdm(proc_data):
    item['answers']['text'] = get_updated_answer(item['context'], item['answers'])

100%|██████████| 10570/10570 [01:45<00:00, 100.24it/s]


In [1279]:
incorrect_answers_no = 0

for index, line in tqdm.tqdm(enumerate(proc_data)):
    try:
        if len(line["answers"]["answer_start"]) == 0:
            continue
        answer_start = line["answers"]["answer_start"][0]
        answer = line["answers"]["text"][0]
        answer_length = len(answer)

        if answer_start == -1:
            continue

        if line["context"][answer_start: answer_start + answer_length] != answer:
            incorrect_answers_no += 1
            print(index, line["context"][answer_start: answer_start + answer_length],
                  "-" * 4, answer)

        # if answer[-1] != ']':
        #     print(index, line["context"][answer_start: answer_start + answer_length], "-" * 4, answer)
        #     incorrect_answers_no += 1

    except Exception as e: 
        print(e)

10570it [00:00, 343680.81it/s]

125 New [nnp] Orleans' [nnp] Mercedes-Benz [nnp] Superdome [nnp] , [sym] Miami [nnp] 's [pos] Sun [nnp] Life [nnp] Stadium [nnp] , [sym] and [cc] the [dt] San [nnp] Francisco [nnp] Bay [nnp] Area [nnp] 's [pos] Levi [nnp] 's [pos] Stadium ---- New [nnp] Orleans' [nnp] Mercedes-Benz [nnp] Superdome [nnp] , [sym] Miami [nnp] 's [pos] Sun [nnp] Life [nnp] Stadium [nnp] , [sym] and [cc] the [dt] San [nnp] Francisco [nnp] Bay [nnp] Area [nnp] 's [pos] Levi [nnp] 's [pos] Stadium
3529 [cd] ⁄ [sym] 3 [cd] normal ---- [cd] ⁄ [sym] 3 [cd] normal
3631 [cd] billion [cd] years [nns] ago [rb] during ---- [cd] billion [cd] years [nns] ago [rb] during
4967 Y. [nnp] p. [nn] orientalis [nn] and [cc] Y. [nnp] p. [nn] medievalis ---- Y. [nnp] p. [nn] orientalis [nn] and [cc] Y. [nnp] p. [nn] medievalis
7145 [prp] was [vbd] never [rb] affiliated [vbn] with [in] any [dt] particular [jj] denomination [nn] , ---- [prp] was [vbd] never [rb] affiliated [vbn] with [in] any [dt] particular [jj] denomination [nn]

In [1280]:
incorrect_answers_no / len(proc_data) * 100

0.056764427625354774

In [1282]:
write_data("./squad_data_validation_pos_ner_answer_updated.json", proc_data)

Data with 'Test' property added has been saved to ./squad_data_validation_pos_ner_answer_updated.json
